In [1]:
article_list = []

for i in range(1, 511):
    if i < 10:
        i = '00' + str(i)
    elif i > 9 and i < 100:
        i = '0' + str(i)
    else:
        i = str(i)
        
    news_article = 'BBC_News_Summary/News_Articles/business/' + i + '.txt'
    news = ''
    with open(news_article) as f:
        for line in f.readlines():
            line = line.strip()
            news += line
            news += " "
        article_list.append(news)

In [2]:
article_list[6]

'Jobs growth still slow in the US  The US created fewer jobs than expected in January, but a fall in jobseekers pushed the unemployment rate to its lowest level in three years.  According to Labor Department figures, US firms added only 146,000 jobs in January. The gain in non-farm payrolls was below market expectations of 190,000 new jobs. Nevertheless it was enough to push down the unemployment rate to 5.2%, its lowest level since September 2001. The job gains mean that President Bush can celebrate - albeit by a very fine margin - a net growth in jobs in the US economy in his first term in office. He presided over a net fall in jobs up to last November\'s Presidential election - the first President to do so since Herbert Hoover. As a result, job creation became a key issue in last year\'s election. However, when adding December and January\'s figures, the administration\'s first term jobs record ended in positive territory.  The Labor Department also said it had revised down the jobs

In [3]:
summary_list = []

for i in range(1, 511):
    if i < 10:
        i = '00' + str(i)
    elif i > 9 and i < 100:
        i = '0' + str(i)
    else:
        i = str(i)
        
    news_article_summary = 'BBC_News_Summary/Summaries/business/' + i + '.txt'
    summary = '_START_ '
    
    with open(news_article_summary) as f:
        for line in f.readlines():
            line = line.strip()
            summary += line
            summary += " "
        summary = summary + " _END_"
        summary_list.append(summary)

In [4]:
summary_list[6]

'_START_ The job gains mean that President Bush can celebrate - albeit by a very fine margin - a net growth in jobs in the US economy in his first term in office.Analysts said the growth in new jobs was not as strong as could be expected given the favourable economic conditions.The Labor Department also said it had revised down the jobs gains in December 2004, from 157,000 to 133,000.The US created fewer jobs than expected in January, but a fall in jobseekers pushed the unemployment rate to its lowest level in three years."The economy is producing a moderate but not a satisfying amount of job growth," said Ken Mayland, president of ClearView Economics.He presided over a net fall in jobs up to last November\'s Presidential election - the first President to do so since Herbert Hoover.  _END_'

In [5]:
import pandas as pd

df = pd.DataFrame(columns = ['article', 'summary'])

for i in range(len(article_list)):
    row = pd.Series({'article' : article_list[i], 'summary' : summary_list[i]})
    df = pd.concat([df, row.to_frame().T], ignore_index = True)
    
df.head()

,article,summary
0,Ad sales boost Time Warner profit Quarterly p...,_START_ TimeWarner said fourth quarter sales r...
1,Dollar gains on Greenspan speech The dollar h...,_START_ The dollar has hit its highest level a...
2,Yukos unit buyer faces loan claim The owners ...,_START_ Yukos' owner Menatep Group says it wil...
3,High fuel prices hit BA's profits British Air...,"_START_ Rod Eddington, BA's chief executive, s..."
4,Pernod takeover talk lifts Domecq Shares in U...,_START_ Pernod has reduced the debt it took on...


In [6]:
from sklearn.model_selection import train_test_split
article_tr, article_val, summary_tr, summary_val=train_test_split(df['article'],
                                                                   df['summary'],
                                                                   test_size=0.1,
                                                                   random_state=0,
                                                                   shuffle=True) 

In [7]:
max_length = 0
articles = list(df['article'])

for article in articles:
    if len(article.split()) > max_length:
        max_length = len(article.split())
    else:
        continue

print(max_length)

891


In [8]:
max_length = 0
summaries = list(df['summary'])

for summary in summaries:
    if len(summary.split()) > max_length:
        max_length = len(summary.split())
    else:
        continue

print(max_length)

405


In [9]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

article_tokenizer = Tokenizer()
article_tokenizer.fit_on_texts(list(df['article']))

article_tr = article_tokenizer.texts_to_sequences(article_tr) 
article_val = article_tokenizer.texts_to_sequences(article_val) 

article_tr = pad_sequences(article_tr,  maxlen = 800, padding='post') 
article_val = pad_sequences(article_val, maxlen = 800, padding='post')

article_voc_size = len(article_tokenizer.word_index) +1

print(article_voc_size)

12675


In [10]:
summary_tokenizer = Tokenizer()
summary_tokenizer.fit_on_texts(list(df['summary']))

summary_tr = summary_tokenizer.texts_to_sequences(summary_tr) 
summary_val = summary_tokenizer.texts_to_sequences(summary_val) 

summary_tr = pad_sequences(summary_tr,  maxlen = 350, padding='post') 
summary_val = pad_sequences(summary_val, maxlen = 350, padding='post')

summary_voc_size = len(summary_tokenizer.word_index) +1

print(summary_voc_size)

8202


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Attention, Normalization, Dense
from tensorflow.keras.layers import Concatenate, TimeDistributed

In [32]:
embedding_inputs = Input(shape=(800), name = 'Encoder_Input') #max length of article
embedding = Embedding(12675, 3, trainable=True, name = 'Encoder_Embedding')(embedding_inputs) #vocab_size_article, neurons

encoder_layer_1 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_1') #neurons 
encoder_output_1, state_h1, state_c1 = encoder_layer_1(embedding)

encoder_layer_2 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_2') #neurons 
encoder_output_2, state_h2, state_c2 = encoder_layer_2(encoder_output_1)

encoder_layer_3 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_3') #neurons 
encoder_output_3, state_h3, state_c3 = encoder_layer_3(encoder_output_2)


decoder_inputs = Input(shape=(None,), name = 'Decoder_Input')
decoder_embedding = Embedding(8202, 50, trainable=True, name = 'Decoder_Embedding')(decoder_inputs)#vocab_size_summary, neurons

decoder_layer_1 = LSTM(32, return_sequences = True, return_state = True, name = 'Decoder_LSTM')
decoder_output_1, decoder_state_h1, decoder_state_c1 = decoder_layer_1(decoder_embedding, 
                                                                       initial_state=[state_h3, state_c3])

attention_layer = Attention()
attn_out, attn_state = attention_layer([encoder_output_3, decoder_output_1], return_attention_scores = True)


decoder_concat = Concatenate(axis=1)([decoder_output_1, attn_out])

decoder_dense = TimeDistributed(Dense(8202, activation = 'softmax'))#vocab_size_summary
decoder_outputs = decoder_dense(decoder_concat)

model = Model([embedding_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Encoder_Input (InputLayer)  [(None, 800)]                0         []                            
                                                                                                  
 Encoder_Embedding (Embeddi  (None, 800, 3)               38025     ['Encoder_Input[0][0]']       
 ng)                                                                                              
                                                                                                  
 LSTM_1 (LSTM)               [(None, 800, 32),            4608      ['Encoder_Embedding[0][0]']   
                              (None, 32),                                                         
                              (None, 32)]                                                   

In [33]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [16]:
labels = summary_tr.reshape(summary_tr.shape[0], summary_tr.shape[1], 1)[:,1:]
labels.shape

(459, 349, 1)

In [17]:
summary_tr[:,:-1].shape

(459, 349)

In [18]:
article_tr.shape

(459, 800)

In [34]:
history = model.fit([article_tr, summary_tr[:,:-1]], 
                    summary_tr.reshape(summary_tr.shape[0], summary_tr.shape[1], 1)[:,1:],
                    epochs=20,
                    batch_size=32)

Epoch 1/20


2023-07-05 14:47:01.903728: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/kawaii/opt/miniconda3/envs/m1_no_gpu/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/kawaii/opt/

# Attention from scratch

In [35]:
from keras.layers import concatenate, Activation, dot

In [89]:
# Loung Attention = normal keras attention 
# the number of neurons should be same for encoder and decoder LSTM, as initial state size should be same 
encoder_inputs = Input(shape=(800), name = 'Encoder_Input') #max length of article
encoder_embedding = Embedding(12675, 100, trainable=True, name = 'Encoder_Embedding')(encoder_inputs)#vocab_size_article, neurons

encoder_layer_1 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_1') #neurons 
encoder_output_1, state_h1, state_c1 = encoder_layer_1(encoder_embedding)

encoder_layer_2 = LSTM(32, return_sequences=True, return_state=True, name = 'LSTM_2') #neurons 
encoder_output_2, state_h2, state_c2 = encoder_layer_2(encoder_output_1)

encoder_layer_3 = LSTM(64, return_sequences=True, return_state=True, name = 'LSTM_3') #neurons 
encoder_output_3, state_h3, state_c3 = encoder_layer_3(encoder_output_2)


decoder_inputs = Input(shape=(None,), name = 'Decoder_Input')
decoder_embedding_layer = Embedding(8202, 100, trainable=True, name = 'Decoder_Embedding')#vocab_size_summary, neurons
decoder_embedding = decoder_embedding_layer(decoder_inputs)

decoder_layer_1 = LSTM(64, return_sequences = True, return_state = True, name = 'Decoder_LSTM')
decoder_output_1, decoder_state_h1, decoder_state_c1 = decoder_layer_1(decoder_embedding, 
                                                                       initial_state=[state_h3, state_c3])

attention_dot = dot([decoder_output_1, encoder_output_3], axes=[2, 2])
attention_layer = Activation('softmax')
attention = attention_layer(attention_dot)

context = dot([attention, encoder_output_3], axes=[2,1])
decoder_combined_context = concatenate([context, decoder_output_1])

decoder_dense = TimeDistributed(Dense(8202, activation = 'softmax'))#vocab_size_summary
decoder_outputs = decoder_dense(decoder_combined_context)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model_31"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Encoder_Input (InputLayer)  [(None, 800)]                0         []                            
                                                                                                  
 Encoder_Embedding (Embeddi  (None, 800, 100)             1267500   ['Encoder_Input[0][0]']       
 ng)                                                                                              
                                                                                                  
 LSTM_1 (LSTM)               [(None, 800, 32),            17024     ['Encoder_Embedding[0][0]']   
                              (None, 32),                                                         
                              (None, 32)]                                                  

In [61]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, show_layer_names=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [90]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [91]:
history = model.fit([article_tr, summary_tr[:,:-1]], 
                    summary_tr.reshape(summary_tr.shape[0], summary_tr.shape[1], 1)[:,1:],
                    epochs=500,
                    batch_size=32)

Epoch 1/500


2023-07-08 01:21:15.124192: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


15/15 [==============================] - 28s 2s/step - loss: 7.1190
Epoch 2/500
15/15 [==============================] - 23s 2s/step - loss: 4.0281
Epoch 3/500
15/15 [==============================] - 26s 2s/step - loss: 3.5184
Epoch 4/500
15/15 [==============================] - 23s 2s/step - loss: 3.1623
Epoch 5/500
15/15 [==============================] - 20s 1s/step - loss: 3.0486
Epoch 6/500
15/15 [==============================] - 18s 1s/step - loss: 2.9963
Epoch 7/500
15/15 [==============================] - 23s 2s/step - loss: 2.9692
Epoch 8/500
15/15 [==============================] - 18s 1s/step - loss: 2.9507
Epoch 9/500
15/15 [==============================] - 18s 1s/step - loss: 2.9326
Epoch 10/500
15/15 [==============================] - 19s 1s/step - loss: 2.9131
Epoch 11/500
15/15 [==============================] - 19s 1s/step - loss: 2.8984
Epoch 12/500
15/15 [==============================] - 18s 1s/step - loss: 2.8870
Epoch 13/500
15/15 [=============================

15/15 [==============================] - 20s 1s/step - loss: 1.9731
Epoch 103/500
15/15 [==============================] - 24s 2s/step - loss: 1.9672
Epoch 104/500
15/15 [==============================] - 21s 1s/step - loss: 1.9575
Epoch 105/500
15/15 [==============================] - 21s 1s/step - loss: 1.9501
Epoch 106/500
15/15 [==============================] - 21s 1s/step - loss: 1.9354
Epoch 107/500
15/15 [==============================] - 21s 1s/step - loss: 1.9290
Epoch 108/500
15/15 [==============================] - 21s 1s/step - loss: 1.9248
Epoch 109/500
15/15 [==============================] - 21s 1s/step - loss: 1.9146
Epoch 110/500
15/15 [==============================] - 21s 1s/step - loss: 1.9079
Epoch 111/500
15/15 [==============================] - 21s 1s/step - loss: 1.8992
Epoch 112/500
15/15 [==============================] - 21s 1s/step - loss: 1.8871
Epoch 113/500
15/15 [==============================] - 23s 2s/step - loss: 1.8806
Epoch 114/500
15/15 [=========

15/15 [==============================] - 22s 1s/step - loss: 1.2362
Epoch 203/500
15/15 [==============================] - 20s 1s/step - loss: 1.2253
Epoch 204/500
15/15 [==============================] - 21s 1s/step - loss: 1.2259
Epoch 205/500
15/15 [==============================] - 22s 1s/step - loss: 1.2167
Epoch 206/500
15/15 [==============================] - 23s 2s/step - loss: 1.2069
Epoch 207/500
15/15 [==============================] - 22s 1s/step - loss: 1.2069
Epoch 208/500
15/15 [==============================] - 24s 2s/step - loss: 1.1999
Epoch 209/500
15/15 [==============================] - 22s 1s/step - loss: 1.1933
Epoch 210/500
15/15 [==============================] - 21s 1s/step - loss: 1.1866
Epoch 211/500
15/15 [==============================] - 21s 1s/step - loss: 1.1822
Epoch 212/500
15/15 [==============================] - 21s 1s/step - loss: 1.1769
Epoch 213/500
15/15 [==============================] - 20s 1s/step - loss: 1.1692
Epoch 214/500
15/15 [=========

15/15 [==============================] - 19s 1s/step - loss: 0.7465
Epoch 303/500
15/15 [==============================] - 20s 1s/step - loss: 0.7458
Epoch 304/500
15/15 [==============================] - 22s 1s/step - loss: 0.7382
Epoch 305/500
15/15 [==============================] - 19s 1s/step - loss: 0.7368
Epoch 306/500
15/15 [==============================] - 20s 1s/step - loss: 0.7325
Epoch 307/500
15/15 [==============================] - 20s 1s/step - loss: 0.7307
Epoch 308/500
15/15 [==============================] - 22s 1s/step - loss: 0.7282
Epoch 309/500
15/15 [==============================] - 21s 1s/step - loss: 0.7238
Epoch 310/500
15/15 [==============================] - 19s 1s/step - loss: 0.7161
Epoch 311/500
15/15 [==============================] - 19s 1s/step - loss: 0.7134
Epoch 312/500
15/15 [==============================] - 19s 1s/step - loss: 0.7117
Epoch 313/500
15/15 [==============================] - 19s 1s/step - loss: 0.7089
Epoch 314/500
15/15 [=========

15/15 [==============================] - 19s 1s/step - loss: 0.4423
Epoch 403/500
15/15 [==============================] - 19s 1s/step - loss: 0.4372
Epoch 404/500
15/15 [==============================] - 19s 1s/step - loss: 0.4338
Epoch 405/500
15/15 [==============================] - 19s 1s/step - loss: 0.4352
Epoch 406/500
15/15 [==============================] - 19s 1s/step - loss: 0.4291
Epoch 407/500
15/15 [==============================] - 19s 1s/step - loss: 0.4265
Epoch 408/500
15/15 [==============================] - 19s 1s/step - loss: 0.4232
Epoch 409/500
15/15 [==============================] - 19s 1s/step - loss: 0.4213
Epoch 410/500
15/15 [==============================] - 19s 1s/step - loss: 0.4199
Epoch 411/500
15/15 [==============================] - 19s 1s/step - loss: 0.4172
Epoch 412/500
15/15 [==============================] - 19s 1s/step - loss: 0.4125
Epoch 413/500
15/15 [==============================] - 19s 1s/step - loss: 0.4172
Epoch 414/500
15/15 [=========

In [83]:
reverse_target_word_index = summary_tokenizer.index_word 
reverse_source_word_index = article_tokenizer.index_word 
target_word_index = summary_tokenizer.word_index

In [84]:
encoder_model = Model(inputs = encoder_inputs, outputs = [encoder_output_3, state_h3, state_c2])

decoder_state_input_h = Input(shape=(32,)) #e_h
decoder_state_input_c = Input(shape=(32,)) #e_c
decoder_hidden_state_input = Input(shape=(800, 32)) #e_out

dec_emb = decoder_embedding_layer(decoder_inputs)
decoder_output_2, decoder_state_h2, decoder_state_c2 = decoder_layer_1(dec_emb, 
                                                       initial_state = [decoder_state_input_h, decoder_state_input_c])

attention_dot = dot([decoder_output_2, decoder_hidden_state_input], axes=[2, 2])
attention = attention_layer(attention_dot)

context = dot([attention, decoder_hidden_state_input], axes=[2,1])
decoder_combined_context = concatenate([context, decoder_output_2])

decoder_outputs = decoder_dense(decoder_combined_context)

decoder_model = Model(
    inputs = [decoder_inputs] + [decoder_hidden_state_input, decoder_state_input_h, decoder_state_input_c],
    outputs = [decoder_outputs] + [decoder_state_h2, decoder_state_c2])

In [85]:
import numpy as np

def decode_sequence(input_seq):
    e_out, e_h, e_c = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))

    # Chose the 'start' word as the first word of the target sequence
    target_seq[0, 0] = target_word_index['start']

    stop_condition = False
    
    decoded_sentence = ''
    
    while not stop_condition:
        print([target_seq])
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]

        if(sampled_token!='end'):
            decoded_sentence += ' '+sampled_token

            # Exit condition: either hit max length or find stop word.
            if (sampled_token == 'end' or len(decoded_sentence.split()) >= (350-1)):
                stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [86]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:      
        if((i != 0 and i != target_word_index['start']) and i != target_word_index['end']):
            newString = newString + reverse_target_word_index[i]+' '
            
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString = newString + reverse_source_word_index[i]+' '
    return newString

In [87]:
#trained for 300 epochs, more embedding neurons
decode_sequence(article_tr[4].reshape(1, 800))

1/1 [==============================] - 1s 532ms/step
[array([[16.]])]
1/1 [==============================] - 0s 161ms/step
[array([[1646.]])]
1/1 [==============================] - 0s 10ms/step
[array([[1646.]])]
1/1 [==============================] - 0s 9ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 10ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 9ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 9ms/step


2023-07-07 17:59:31.031665: W tensorflow/core/grappler/costs/op_level_cost_estimator.cc:693] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" model: "0" frequency: 2400 num_cores: 8 environment { key: "cpu_instruction_set" value: "ARM NEON" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 16384 l2_cache_size: 524288 l3_cache_size: 524288 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


[array([[4090.]])]
1/1 [==============================] - 0s 10ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 9ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 9ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 9ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 12ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 10ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 9ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 10ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1629.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1646.]])]
1/1 [==============================] - 0s 10ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 9ms/step
[array([[824.]])]
1/1 [==============================] - 0s 9ms/step
[array([[27.]])]
1/1 [==============================] - 0s 9ms/step
[array([[107.]])]


1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 12ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 10ms/step
[array([[

1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.

" w w sontaya sontaya sontaya sontaya sontaya sontaya sontaya sontaya sontaya sontaya sontaya sontaya berlusconi w sontaya brown have who who have been have said that the kok's are korea with financial corporate governance and the bank of the bush's bank of the bush's year the bank of england costs were set for the year and the bank and financial operations in the world market in addition the us access to take the us nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun

In [75]:
reverse_target_word_index[0] = 'nun'

In [77]:
#trained for 50 epochs, less embedding neuron
decode_sequence(article_tr[5].reshape(1, 800))

1/1 [==============================] - 0s 55ms/step
[array([[16.]])]
1/1 [==============================] - 0s 9ms/step
[array([[108.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 8ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 10ms/step
[array([[1069.]])]
1/1 [==============================] - 0s 9ms/step
[array([[26.]])]
1/1 [==============================] - 0s 9ms/step
[array([[805.]])]
1/1 [==============================] - 0s 10ms/step
[array([[1.]])]
1/1 [=====================

1/1 [==============================] - 0s 8ms/step
[array([[4.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1.]])]
1/1 [==============================] - 0s 9ms/step
[array([[148.]])]
1/1 [==============================] - 0s 9ms/step
[array([[102.]])]
1/1 [==============================] - 0s 9ms/step
[array([[44.]])]
1/1 [==============================] - 0s 8ms/step
[array([[1.]])]
1/1 [==============================] - 0s 9ms/step
[array([[48.]])]
1/1 [==============================] - 0s 9ms/step
[array([[7.]])]
1/1 [==============================] - 0s 8ms/step
[array([[1.]])]
1/1 [==============================] - 0s 9ms/step
[array([[48.]])]
1/1 [==============================] - 0s 9ms/step
[array([[7.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1.]])]
1/1 [==============================] - 0s 9ms/step
[array([[48.]])]
1/1 [==============================] - 0s 8ms/step
[array([[4.]])]
1/1 [==============================] - 0s 9ms/step
[ar

1/1 [==============================] - 0s 9ms/step
[array([[44.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1.]])]
1/1 [==============================] - 0s 9ms/step
[array([[48.]])]
1/1 [==============================] - 0s 9ms/step
[array([[7.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1.]])]
1/1 [==============================] - 0s 9ms/step
[array([[48.]])]
1/1 [==============================] - 0s 9ms/step
[array([[7.]])]
1/1 [==============================] - 0s 9ms/step
[array([[1.]])]
1/1 [==============================] - 0s 9ms/step
[array([[48.]])]
1/1 [==============================] - 0s 9ms/step
[array([[4.]])]
1/1 [==============================] - 0s 8ms/step
[array([[1.]])]
1/1 [==============================] - 0s 9ms/step
[array([[148.]])]
1/1 [==============================] - 0s 9ms/step
[array([[102.]])]
1/1 [==============================] - 0s 9ms/step
[array([[44.]])]
1/1 [==============================] - 0s 9ms/step
[a

" month greenspan greenspan greenspan greenspan greenspan greenspan greenspan greenspan greenspan year april the year greenspan the executive the executive the executive the executive the sullivan smith lecture to greenspan to a statement mr executive mr sullivan smith who said a former executive mr executive mr executive mr brown said the second largest market the bush that he said that the biggest bank of oil cuts with the world's biggest economy the world's largest economy committee the bank said the bank of the world's biggest economy the bank said the bank said the bank of the world's biggest economy the bank said the bank said the bank of the world's biggest economy the bank said the bank said the bank of the world's biggest economy the bank said the bank said the bank of the world's biggest economy the bank said the bank said the bank of the world's biggest economy the bank said the bank said the bank of the world's biggest economy the bank said the bank said the bank of the wor

In [92]:
#trained for 500 epochs, 100 embedding neuron/dimension
decode_sequence(article_tr[5].reshape(1, 800))

1/1 [==============================] - 0s 132ms/step
[array([[16.]])]
1/1 [==============================] - 0s 34ms/step
[array([[1646.]])]
1/1 [==============================] - 0s 11ms/step
[array([[1646.]])]
1/1 [==============================] - 0s 10ms/step
[array([[1646.]])]
1/1 [==============================] - 0s 11ms/step
[array([[734.]])]
1/1 [==============================] - 0s 11ms/step
[array([[4.]])]
1/1 [==============================] - 0s 11ms/step
[array([[824.]])]
1/1 [==============================] - 0s 12ms/step
[array([[4.]])]
1/1 [==============================] - 0s 10ms/step
[array([[824.]])]
1/1 [==============================] - 0s 10ms/step
[array([[2.]])]
1/1 [==============================] - 0s 11ms/step
[array([[1053.]])]
1/1 [==============================] - 0s 10ms/step
[array([[4.]])]
1/1 [==============================] - 0s 11ms/step
[array([[4090.]])]
1/1 [==============================] - 0s 10ms/step
[array([[6999.]])]
1/1 [=================

1/1 [==============================] - 0s 11ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0

1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 10ms/step
[array([[0.]])]
1/1 [==============================] - 0s 8ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 10ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 10ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([[0.]])]
1/1 [==============================] - 0s 9ms/step
[array([

" w w w yen of brown of brown to manufacturers of sontaya islands weather travellers to top wages delivery swedish executive to brown to the tellier's of the us market and the company said the world's biggest car market maker in oil group a slight price of oil prices have been up by a barrel to be for the us and market and boost a barrel for the country's tax nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nun nu

In [93]:
seq2text(article_tr[5])

"peugeot deal boosts mitsubishi struggling japanese car maker mitsubishi motors has struck a deal to supply french car maker peugeot with 30 000 sports utility vehicles suv the two firms signed a memorandum of understanding and say they expect to seal a final agreement by spring 2005 the alliance comes as a badly needed boost for loss making mitsubishi after several profit warnings and poor sales the suvs will be built in japan using peugeot's diesel engines and sold mainly in the european market falling sales have left mitsubishi motors with underused capacity and the production deal with peugeot gives it a chance to utilise some of it in january mitsubishi motors issued its third profits warning in nine months and cut its sales forecasts for the year to march 2005 its sales have slid 41 in the past year catalysed by the revelation that the company had systematically been hiding records of faults and then secretly repairing vehicles as a result the japanese car maker has sought a seri

In [94]:
seq2text(summary_tr[5])

'at away rising recent very your risen is slid a expected to trillion big recent very deposit be german over options operate trial revelation your risen is accounts a remain states in stated travel of by say into year to used business company stock has account a likely unemployment be rising lost models here to trillion has be citizens over 9bn grew for exchange of airline of off your risen estimates by buy made enter of signs december and bid by after accused for the year to used business selling after mr stronger your risen be amounting 53 and the likely expected be deposit creating has a governance to chamber three in has us a compared the rising recent very is balance a concerned in into £100m its '